## Антагонистическая игра 5x5

Проблему можно формализовать как задачу оптимизации:
$$V\to\max,$$
при ограничениях
$$P^{\mathrm{T}} X \geq V,\ 1^{\mathrm{T}}X = 1,\ X\geq 0.$$

Переместим $V$ в левую часть уравнения,
введём переменные дополняющей нежёсткости
и получим следующую линейную программу:

$$
\left\{
\begin{aligned}
& {-}V\to\inf,\\
&
\left(
\begin{array}{c|c|c}
\begin{matrix}
-1 \\ \vdots \\ -1
\end{matrix}
&
\begin{matrix}
-1 & & 0 \\
   & \ddots & \\
0  & & -1
\end{matrix} &
P^{\mathrm{T}} \\ \hline
\begin{matrix}
0
\end{matrix}
&
\begin{matrix}
0 & \cdots & 0
\end{matrix}
&
\begin{matrix}
1 & \cdots & 1
\end{matrix}
\end{array}
\right)
\begin{pmatrix}
V\\
s_1\\
\vdots\\
s_n\\
x_1\\
\vdots\\
x_n
\end{pmatrix}
=\begin{pmatrix}
0\\
\vdots\\
0\\ \hline
1
\end{pmatrix}
,\\
& V, s, X \geq 0.
\end{aligned}
\right.$$

<img width="40%" src="1.jpg" alt="problem 1 statement" />

In [1]:
! cat -n P1.txt

     1	.5 .5 .4 .5 .201
     2	.5 .4 .7 .101 .6
     3	.201 .3 .4 .1 .7
     4	.3 .6 .7 .3 .2
     5	.4 .4 .3 .01 .2


In [2]:
function [X, V] = solve_antagonistic (P)
    [m, n] = size(P);
    A = [-ones(n,1), -eye(n), P'; zeros(1,n+1), ones(1,m)];
    b = [zeros(n,1);1];
    c = [1; zeros(size(A,2)-1,1)];
    X = glpk(-c,A,b);
    V = c'*X;
    X = X(n+2:end);
endfunction

function [X,V] = solve_1 ()
    P = dlmread('P1.txt');
    [X, V] = solve_antagonistic(P);
endfunction

solve_1

ans =

   0.66741
   0.00000
   0.33259
   0.00000
   0.00000



**Следует выбрать товары типов $1$ и $3$ в соотношении $2:1$**